In [338]:
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier

In [339]:
matches = pd.read_csv("version2.csv")

In [340]:
matches.columns

Index(['hBall possession', 'aBall possession', 'hExpected goals',
       'aExpected goals', 'hBig chances', 'aBig chances', 'hTotal shots',
       'aTotal shots', 'hGoalkeeper saves', 'aGoalkeeper saves',
       'hCorner kicks', 'aCorner kicks', 'hFouls', 'aFouls', 'hPasses',
       'aPasses', 'hTackles', 'aTackles', 'hFree kicks', 'aFree kicks',
       'hYellow cards', 'aYellow cards', 'hRed cards', 'aRed cards', 'hOdds',
       'aOdds', 'hTeam', 'aTeam', 'hScore', 'aScore', 'hWin', 'dateTime'],
      dtype='object')

In [341]:
matches.shape

(289, 32)

In [342]:

odds = matches['hOdds']
nOdds = []
for odd in odds:
    split = odd.split("/")
    nOdds.append(round(int(split[0]) / int(split[1]),2))

matches['h_nOdds'] = nOdds


odds = matches['aOdds']
nOdds = []
for odd in odds:
    split = odd.split("/")
    nOdds.append(round(int(split[0]) / int(split[1]),2))

matches['a_nOdds'] = nOdds


In [343]:
matches['FdateTime'] = pd.to_datetime(matches['dateTime'])

In [344]:
matches['hTeamCode'] = matches['hTeam'].astype('category').cat.codes

In [345]:
matches['aTeamCode'] = matches['aTeam'].astype('category').cat.codes

In [346]:
matches['dateTime'] = pd.to_datetime(matches['dateTime'])
matches['dateCode'] = matches['dateTime'].dt.strftime('%Y%m%d').astype(int)

In [347]:
matches['timeCode'] = matches['dateTime'].dt.strftime('%H%m%s').astype(int)

In [348]:
matches.drop('FdateTime', axis = "columns")

,hBall possession,aBall possession,hExpected goals,aExpected goals,hBig chances,aBig chances,hTotal shots,aTotal shots,hGoalkeeper saves,aGoalkeeper saves,...,hScore,aScore,hWin,dateTime,h_nOdds,a_nOdds,hTeamCode,aTeamCode,dateCode,timeCode
0,39%,61%,0.72,3.43,2,4,11,37,6,7,...,0,2,0,2025-01-18 15:00:00+00:00,5.00,0.50,3,11,20250118,15011737212400
1,29%,71%,0.31,1.99,1,4,6,23,5,2,...,1,1,0,2025-01-14 20:00:00+00:00,4.00,0.62,15,11,20250114,20011736884800
2,52%,48%,1.04,0.74,1,2,16,11,1,4,...,1,1,0,2024-08-31 14:00:00+00:00,1.00,2.70,15,19,20240831,14081725109200
3,51%,49%,2.44,1.86,3,4,16,12,1,2,...,4,1,1,2024-11-03 14:00:00+00:00,1.25,1.80,17,1,20241103,14111730642400
4,32%,68%,1.52,1.62,2,2,10,17,5,5,...,1,1,0,2025-03-09 16:30:00+00:00,3.50,0.75,13,0,20250309,16031741537800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,66%,34%,2.09,0.63,4,1,24,8,2,4,...,1,1,0,2024-12-26 12:30:00+00:00,0.25,10.00,12,7,20241226,12121735216200
285,50%,50%,1.76,0.97,3,0,20,20,7,5,...,1,0,1,2024-10-21 19:00:00+00:00,1.30,1.90,15,6,20241021,19101729533600
286,42%,58%,0.84,2.44,3,6,9,18,4,4,...,0,5,0,2024-12-15 19:00:00+00:00,0.00,0.00,16,17,20241215,19121734289200
287,40%,60%,1.54,3.60,2,5,12,16,2,3,...,2,5,0,2024-11-30 17:30:00+00:00,4.75,0.50,18,0,20241130,17111732987800


In [349]:
rf = RandomForestClassifier (n_estimators = 50, min_samples_split = 10, random_state = 1)

In [350]:
matches['betFail'] = np.where(
    ((matches['hWin'] == 0) & (matches['h_nOdds'] < 1)) |
    ((matches['hWin'] == 1) & (matches['h_nOdds'] > 1)),
    0,  
    1   
)
matches['aWin'] = np.where(
    ((matches['hScore'] < matches['aScore'])), 1,0
)

In [351]:
matches

,hBall possession,aBall possession,hExpected goals,aExpected goals,hBig chances,aBig chances,hTotal shots,aTotal shots,hGoalkeeper saves,aGoalkeeper saves,...,dateTime,h_nOdds,a_nOdds,FdateTime,hTeamCode,aTeamCode,dateCode,timeCode,betFail,aWin
0,39%,61%,0.72,3.43,2,4,11,37,6,7,...,2025-01-18 15:00:00+00:00,5.00,0.50,2025-01-18 15:00:00+00:00,3,11,20250118,15011737212400,1,1
1,29%,71%,0.31,1.99,1,4,6,23,5,2,...,2025-01-14 20:00:00+00:00,4.00,0.62,2025-01-14 20:00:00+00:00,15,11,20250114,20011736884800,1,0
2,52%,48%,1.04,0.74,1,2,16,11,1,4,...,2024-08-31 14:00:00+00:00,1.00,2.70,2024-08-31 14:00:00+00:00,15,19,20240831,14081725109200,1,0
3,51%,49%,2.44,1.86,3,4,16,12,1,2,...,2024-11-03 14:00:00+00:00,1.25,1.80,2024-11-03 14:00:00+00:00,17,1,20241103,14111730642400,0,0
4,32%,68%,1.52,1.62,2,2,10,17,5,5,...,2025-03-09 16:30:00+00:00,3.50,0.75,2025-03-09 16:30:00+00:00,13,0,20250309,16031741537800,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,66%,34%,2.09,0.63,4,1,24,8,2,4,...,2024-12-26 12:30:00+00:00,0.25,10.00,2024-12-26 12:30:00+00:00,12,7,20241226,12121735216200,0,0
285,50%,50%,1.76,0.97,3,0,20,20,7,5,...,2024-10-21 19:00:00+00:00,1.30,1.90,2024-10-21 19:00:00+00:00,15,6,20241021,19101729533600,0,0
286,42%,58%,0.84,2.44,3,6,9,18,4,4,...,2024-12-15 19:00:00+00:00,0.00,0.00,2024-12-15 19:00:00+00:00,16,17,20241215,19121734289200,0,1
287,40%,60%,1.54,3.60,2,5,12,16,2,3,...,2024-11-30 17:30:00+00:00,4.75,0.50,2024-11-30 17:30:00+00:00,18,0,20241130,17111732987800,1,1


In [352]:
predictors = ['h_nOdds','hTeamCode','aTeamCode','dateCode','timeCode','hWin','a_nOdds']
X = matches[predictors]
y = matches['betFail']

In [353]:
from sklearn.model_selection import train_test_split

In [354]:
X_train,X_test,y_train,y_test = train_test_split (X,y,random_state = 0)

In [355]:
rf.fit (X_train,y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [356]:
preds = rf.predict(X_test)

In [357]:
acc = accuracy_score(y_test,preds)

In [358]:
acc

0.7534246575342466

In [359]:
precision_score(y_test,preds)

0.76

In [360]:
grouped_matches = matches.groupby('hTeam')

In [361]:
def rolling_averages (group,cols,new_cols):
    group = group.sort_values('timeCode')
    rolling_stats = group[cols].rolling(3,closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    group = get_form(group)
    return group

def weighted_form(arr):
    weights = np.arange(1, len(arr) + 1)
    
    ## need to account for larger match form 

    return np.sum(arr * weights) / np.sum(weights)
    
## use neural networks 
def get_form (group):
    group['hForm'] = group['hWin'].rolling(3,closed='left').apply(weighted_form, raw=True)
    group['aForm'] = group['aWin'].rolling(3,closed='left').apply(weighted_form, raw=True)
    group = group.dropna(subset=['hForm','aForm'])
    return group

In [362]:
matches.columns

Index(['hBall possession', 'aBall possession', 'hExpected goals',
       'aExpected goals', 'hBig chances', 'aBig chances', 'hTotal shots',
       'aTotal shots', 'hGoalkeeper saves', 'aGoalkeeper saves',
       'hCorner kicks', 'aCorner kicks', 'hFouls', 'aFouls', 'hPasses',
       'aPasses', 'hTackles', 'aTackles', 'hFree kicks', 'aFree kicks',
       'hYellow cards', 'aYellow cards', 'hRed cards', 'aRed cards', 'hOdds',
       'aOdds', 'hTeam', 'aTeam', 'hScore', 'aScore', 'hWin', 'dateTime',
       'h_nOdds', 'a_nOdds', 'FdateTime', 'hTeamCode', 'aTeamCode', 'dateCode',
       'timeCode', 'betFail', 'aWin'],
      dtype='object')

In [363]:
cols = 'hBall possession,aBall possession,hExpected goals,aExpected goals,hBig chances,aBig chances,hTotal shots,aTotal shots,hGoalkeeper saves,aGoalkeeper saves,hCorner kicks,aCorner kicks,hFouls,aFouls,hPasses,aPasses,hTackles,aTackles,hFree kicks,aFree kicks,hYellow cards,aYellow cards,aRed cards,hRed cards,hScore,aScore'.split(",")

In [364]:
new_cols = [f"{c}_rolling" for c in cols]
#rolling_averages (group,cols,new_cols)


In [365]:
matches['hBall possession'] = matches['hBall possession'].str.rstrip('%').astype(float) / 100
matches['aBall possession'] = matches['aBall possession'].str.rstrip('%').astype(float) / 100

In [366]:
matches_rolling = matches.groupby('hTeam').apply (lambda x: rolling_averages(x,cols,new_cols))

/var/folders/_x/r1xjhn255nq0l1y6fggns8km0000gp/T/ipykernel_4337/3331675197.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby('hTeam').apply (lambda x: rolling_averages(x,cols,new_cols))


In [367]:
matches_rolling.columns

Index(['hBall possession', 'aBall possession', 'hExpected goals',
       'aExpected goals', 'hBig chances', 'aBig chances', 'hTotal shots',
       'aTotal shots', 'hGoalkeeper saves', 'aGoalkeeper saves',
       'hCorner kicks', 'aCorner kicks', 'hFouls', 'aFouls', 'hPasses',
       'aPasses', 'hTackles', 'aTackles', 'hFree kicks', 'aFree kicks',
       'hYellow cards', 'aYellow cards', 'hRed cards', 'aRed cards', 'hOdds',
       'aOdds', 'hTeam', 'aTeam', 'hScore', 'aScore', 'hWin', 'dateTime',
       'h_nOdds', 'a_nOdds', 'FdateTime', 'hTeamCode', 'aTeamCode', 'dateCode',
       'timeCode', 'betFail', 'aWin', 'hBall possession_rolling',
       'aBall possession_rolling', 'hExpected goals_rolling',
       'aExpected goals_rolling', 'hBig chances_rolling',
       'aBig chances_rolling', 'hTotal shots_rolling', 'aTotal shots_rolling',
       'hGoalkeeper saves_rolling', 'aGoalkeeper saves_rolling',
       'hCorner kicks_rolling', 'aCorner kicks_rolling', 'hFouls_rolling',
       'aFouls

In [368]:
matches_rolling.droplevel('hTeam')

,hBall possession,aBall possession,hExpected goals,aExpected goals,hBig chances,aBig chances,hTotal shots,aTotal shots,hGoalkeeper saves,aGoalkeeper saves,...,hFree kicks_rolling,aFree kicks_rolling,hYellow cards_rolling,aYellow cards_rolling,aRed cards_rolling,hRed cards_rolling,hScore_rolling,aScore_rolling,hForm,aForm
197,0.66,0.34,1.19,0.33,1,0,19,7,0,3,...,10.000000,12.333333,1.000000,3.333333,0.000000,0.333333,2.333333,1.333333,0.500000,0.500000
155,0.77,0.23,1.22,0.09,1,0,13,2,0,5,...,11.666667,12.000000,1.333333,2.666667,0.000000,0.333333,2.000000,0.666667,0.666667,0.333333
200,0.46,0.54,1.00,0.81,2,1,12,7,3,2,...,11.666667,10.666667,1.333333,2.666667,0.000000,0.333333,1.000000,0.333333,0.333333,0.166667
278,0.45,0.55,0.92,0.81,2,2,9,9,2,1,...,9.000000,7.333333,1.666667,1.666667,0.000000,0.000000,2.666667,0.333333,0.666667,0.000000
186,0.66,0.34,1.33,1.04,2,2,18,8,2,4,...,10.000000,8.666667,1.333333,1.666667,0.000000,0.000000,2.333333,1.000000,0.333333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,0.57,0.43,1.53,2.51,4,4,11,19,4,4,...,10.000000,16.000000,2.000000,3.000000,0.000000,0.333333,1.333333,1.333333,0.500000,0.500000
117,0.51,0.49,0.86,0.44,1,0,7,11,4,2,...,10.000000,14.000000,1.666667,2.666667,0.000000,0.000000,1.666667,1.333333,0.333333,0.166667
172,0.60,0.40,1.31,1.69,0,4,18,11,3,4,...,11.333333,13.000000,1.666667,3.000000,0.333333,0.000000,2.000000,0.666667,0.666667,0.000000
130,0.61,0.39,1.59,2.13,2,4,13,11,0,5,...,12.000000,9.000000,1.333333,3.000000,0.333333,0.000000,1.666667,1.333333,0.333333,0.500000


In [373]:
def make_prediction (data,predictors):
    rf = RandomForestClassifier (n_estimators = 50, min_samples_split = 10, random_state = 1)
    X = data[predictors]
    y = data['betFail']
    X_train,X_test,y_train,y_test = train_test_split (X,y,random_state = 0)
    rf.fit (X_train,y_train)
    preds = rf.predict(X_test)
    acc = accuracy_score (y_test,preds)
    precision = precision_score (y_test,preds)
    print (acc,precision)

In [377]:
new_predictors = predictors + new_cols + ['hForm','aForm']
make_prediction (matches_rolling,new_predictors)
#new_predictors

0.6046511627906976 0.6666666666666666


In [371]:
matches_rolling.columns

Index(['hBall possession', 'aBall possession', 'hExpected goals',
       'aExpected goals', 'hBig chances', 'aBig chances', 'hTotal shots',
       'aTotal shots', 'hGoalkeeper saves', 'aGoalkeeper saves',
       'hCorner kicks', 'aCorner kicks', 'hFouls', 'aFouls', 'hPasses',
       'aPasses', 'hTackles', 'aTackles', 'hFree kicks', 'aFree kicks',
       'hYellow cards', 'aYellow cards', 'hRed cards', 'aRed cards', 'hOdds',
       'aOdds', 'hTeam', 'aTeam', 'hScore', 'aScore', 'hWin', 'dateTime',
       'h_nOdds', 'a_nOdds', 'FdateTime', 'hTeamCode', 'aTeamCode', 'dateCode',
       'timeCode', 'betFail', 'aWin', 'hBall possession_rolling',
       'aBall possession_rolling', 'hExpected goals_rolling',
       'aExpected goals_rolling', 'hBig chances_rolling',
       'aBig chances_rolling', 'hTotal shots_rolling', 'aTotal shots_rolling',
       'hGoalkeeper saves_rolling', 'aGoalkeeper saves_rolling',
       'hCorner kicks_rolling', 'aCorner kicks_rolling', 'hFouls_rolling',
       'aFouls

In [372]:
matches_rolling.to_csv('df.csv',index=False)